Run `pip install .` first to install all dependencies.

In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score

import sys

sys.path.append("..")
from otc.models.objective import set_seed
from otc.data.fs import fs
from otc.models.classical_classifier import ClassicalClassifier


In [2]:
print(fs.buckets)


['thesis-bucket-option-trade-classification/']


In [3]:
# set seed for reproducability
seed = set_seed()


In [4]:
columns = [
    "buy_sell",
    "EXPIRATION",
    "QUOTE_DATETIME",
    "TRADE_PRICE",
    "price_ex_lag",
    "price_all_lag",
    "price_ex_lead",
    "price_all_lead",
    "TRADE_SIZE",
    "ask_size_ex",
    "bid_size_ex",
    "BEST_BID",
    "BEST_ASK",
    "bid_ex",
    "ask_ex",
]

train = pd.read_parquet(
    f"gs://thesis-bucket-option-trade-classification/data/preprocessed/train_set_60.parquet",
    engine="fastparquet",
    columns=columns,
)
val = pd.read_parquet(
    f"gs://thesis-bucket-option-trade-classification/data/preprocessed/val_set_20.parquet",
    engine="fastparquet",
    columns=columns,
)
test = pd.read_parquet(
    f"gs://thesis-bucket-option-trade-classification/data/preprocessed/test_set_20.parquet",
    engine="fastparquet",
    columns=columns,
)

X_print = pd.concat([train, val, test])


/usr/local/lib/python3.8/dist-packages/google/auth/_default.py:83: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


### Robustness Checks

In [5]:
# add baseline results
X_print["rule"] = "Baseline"
X_print["buy_sell_predicted"] = 0

# # prepare columns for printing
# self.X_["ttm"] = (
#     self.X_["EXPIRATION"].dt.to_period("M")
#     - self.X_["QUOTE_DATETIME"].dt.to_period("M")
# ).apply(lambda x: x.n)

X_print["year"] = X_print["QUOTE_DATETIME"].dt.year

bins_tradesize = [-1, 1, 3, 5, 11, np.inf]
trade_size_labels = ["(0,1]", "(1,3]", "(3,5]", "(5,11]", ">11"]
X_print["TRADE_SIZE_binned"] = pd.cut(
    X_print["TRADE_SIZE"], bins_tradesize, labels=trade_size_labels
)

bins_years = [2004, 2007, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]
year_labels = [
    "2005-2007",
    "2008-2010",
    "2011",
    "2012",
    "2013",
    "2014",
    "2015",
    "2016",
    "2017",
]
X_print["year_binned"] = pd.cut(X_print["year"], bins_years, labels=year_labels)

bins_ttm = [-1, 1, 2, 3, 6, 12, np.inf]
ttm_labels = [
    "ttm <= 1 month",
    "ttm (1-2] month",
    "ttm (2-3] month",
    "ttm (3-6] month",
    "ttm (6-12] month",
    "ttm > 12 month",
]
# X_print["ttm_binned"] = pd.cut(X_print["ttm"], bins_ttm, labels=ttm_labels)

# TODO: Security type
# TODO: Moneyness
# TODO: time from previous trade; same underlying or any?


In [6]:
def check_robustness(criterion: str = "year_binned") -> pd.DataFrame:
    """
    Check robustness of rules by calculating the accuracy for a given
    criterion and rules.

    Example:
    rule		Baseline
    TRADE_SIZE_binned
    (0,1]	  0.710966
    (1,3]	  0.717664
    (3,5]	  0.715195
    (5,11]	0.699428
    >11	  	0.688348

    Args:
        criterion (str, optional): criterion to check robustness for.
        Defaults to "year_binned".

    Returns:
        pd.DataFrame: DataFrame with accuracy of rules. Rule in columns and
        criterion values in rows.
    """

    # fill others randomly with equal weights
    X_print["buy_sell_predicted"] = X_print["buy_sell_predicted"].map(
        lambda l: l if not np.isnan(l) else np.random.choice([-1, 1])
    )

    # cuculate average over columns if multiple subsets are combined
    results = (
        X_print.groupby(["rule", criterion])[["buy_sell", "buy_sell_predicted"]]
        .apply(lambda x: accuracy_score(x["buy_sell"], x["buy_sell_predicted"]))
        .unstack(level=0)
        # .assign(avg=lambda x: x.mean(axis=1))
        .mul(100)
    )
    return results


In [7]:
def combine_results(revised: pd.DataFrame, base: pd.DataFrame) -> pd.DataFrame:
    """
    Generate print layout like in Grauer et al.

    Example:
    TRADE_SIZE_binned	(0,1]	(1,3]	(3,5]	(5,11]	>11
    rule
    tick rule (all)	62.29 (10.38)	62.91 (10.27)	63.54 (11.08)	58.64 (5.62)	55.41 (-0.94)
    """
    combo = base.copy()
    for i, col in enumerate(combo.columns):
        col_other = revised.columns[i]
        combo[col] = (
            revised[col_other].round(2).astype(str)
            + " ("
            + (revised[col_other] - base[col]).round(2).astype(str)
            + ")"
        )
    return combo.T


## Classical Rules

### Tick Rule

In [8]:
clf = ClassicalClassifier(
    layers=[
        ("trade_size", "ex"),
        ("tick", "all"),
    ],
    random_state=seed,
)
clf.fit(X=X_print.loc[0:1, :], y=X_print["buy_sell"].loc[0:1])
X_print["buy_sell_predicted"] = clf.predict(X_print)
X_print["rule"] = "tradesize + tick (all)"

result = check_robustness("TRADE_SIZE_binned")
result


rule,tradesize + tick (all)
TRADE_SIZE_binned,
"(0,1]",62.344979
"(1,3]",63.012207
"(3,5]",63.612356
"(5,11]",58.701591
>11,55.441527


In [9]:
# see p. 30
np.testing.assert_allclose(
    result.iloc[:, 0].tolist(), [62.29, 62.92, 63.54, 58.64, 55.42], atol=0.5
)


In [10]:
# TODO: result is not consistent with p. 36;
# Group result on p. 35 is not consistent with p. 31.
result = check_robustness("year_binned")
result


rule,tradesize + tick (all)
year_binned,
2005-2007,64.478336
2008-2010,61.403698
2011,58.807349
2012,58.727782
2013,59.791531
2014,62.262026
2015,58.650873
2016,58.414971
2017,59.195714


In [11]:
# see p. 36
np.testing.assert_allclose(
    result.iloc[:, 0].tolist(),
    [65.44, 62.26, 59.18, 58.94, 59.74, 61.87, 58.49, 58.48, 59.36],
    atol=1.0,
)


In [12]:
clf = ClassicalClassifier(
    layers=[
        ("trade_size", "ex"),
        ("tick", "ex"),
    ],
    random_state=seed,
)
clf.fit(X=X_print.loc[0:1, :], y=X_print["buy_sell"].loc[0:1])
X_print["buy_sell_predicted"] = clf.predict(X_print)
X_print["rule"] = "tradesize + tick (all)"

result = check_robustness("TRADE_SIZE_binned")
result


rule,tradesize + tick (all)
TRADE_SIZE_binned,
"(0,1]",59.203776
"(1,3]",59.717659
"(3,5]",60.103457
"(5,11]",55.583810
>11,51.631437


### Reverse Tick Rule

In [13]:
clf = ClassicalClassifier(
    layers=[
        ("trade_size", "ex"),
        ("rev_tick", "all"),
    ],
    random_state=seed,
)
clf.fit(X=X_print.loc[0:1, :], y=X_print["buy_sell"].loc[0:1])
X_print["buy_sell_predicted"] = clf.predict(X_print)
X_print["rule"] = "tradesize + tick (all)"

result = check_robustness("TRADE_SIZE_binned")
result


rule,tradesize + tick (all)
TRADE_SIZE_binned,
"(0,1]",63.129104
"(1,3]",63.532943
"(3,5]",64.520294
"(5,11]",59.128625
>11,54.608586


In [14]:
# see p. 30
np.testing.assert_allclose(
    result.iloc[:, 0].tolist(), [63.51, 63.87, 64.94, 59.53, 55.05], atol=0.5
)


### Quote Rule

In [15]:
clf = ClassicalClassifier(
    layers=[
        ("trade_size", "ex"),
        ("quote", "best"),
        ("quote", "ex"),
    ],
    random_state=seed,
)
clf.fit(X=X_print.loc[0:1, :], y=X_print["buy_sell"].loc[0:1])
X_print["buy_sell_predicted"] = clf.predict(X_print)
X_print["rule"] = "Tradesize + Quote (NBBO) + Quote (ISE)"

result = check_robustness("TRADE_SIZE_binned")
result


rule,Tradesize + Quote (NBBO) + Quote (ISE)
TRADE_SIZE_binned,
"(0,1]",74.944263
"(1,3]",78.279335
"(3,5]",79.981987
"(5,11]",72.592750
>11,69.586900


In [16]:
# see p. 30
np.testing.assert_allclose(
    result.iloc[:, 0].tolist(), [74.94, 78.28, 79.97, 72.59, 69.58], atol=0.5
)


### LR Algorithm

In [17]:
clf = ClassicalClassifier(
    layers=[
        ("trade_size", "ex"),
        ("lr", "best"),
    ],
    random_state=seed,
)
clf.fit(X=X_print.loc[0:1, :], y=X_print["buy_sell"].loc[0:1])
X_print["buy_sell_predicted"] = clf.predict(X_print)
X_print["rule"] = "Tradesize + LR (NBBO)"

result = check_robustness("TRADE_SIZE_binned")
result


rule,Tradesize + LR (NBBO)
TRADE_SIZE_binned,
"(0,1]",74.434941
"(1,3]",77.766009
"(3,5]",79.458024
"(5,11]",72.208690
>11,69.256442


In [19]:
# see p. 30
np.testing.assert_allclose(
    result.iloc[:, 0].tolist(), [74.09, 77.31, 78.96, 71.57, 68.40], atol=1
)


### Reverse LR Algorithm

In [20]:
clf = ClassicalClassifier(
    layers=[
        ("trade_size", "ex"),
        ("rev_lr", "best"),
    ],
    random_state=seed,
)
clf.fit(X=X_print.loc[0:1, :], y=X_print["buy_sell"].loc[0:1])
X_print["buy_sell_predicted"] = clf.predict(X_print)
X_print["rule"] = "Tradesize + rev. LR (NBBO)"

result_ts = check_robustness("TRADE_SIZE_binned")
result_ts


rule,Tradesize + rev. LR (NBBO)
TRADE_SIZE_binned,
"(0,1]",74.592932
"(1,3]",77.927223
"(3,5]",79.656313
"(5,11]",72.351442
>11,69.327030


In [21]:
# see p. 30
np.testing.assert_allclose(
    result_ts.iloc[:, 0].tolist(), [74.64, 77.95, 79.68, 72.38, 69.33], atol=0.5
)


In [22]:
clf = ClassicalClassifier(
    layers=[
        ("rev_lr", "best"),
    ],
    random_state=seed,
)
clf.fit(X=X_print.loc[0:1, :], y=X_print["buy_sell"].loc[0:1])
X_print["buy_sell_predicted"] = clf.predict(X_print)
X_print["rule"] = "reverse LR (NBBO)"

result = check_robustness("TRADE_SIZE_binned")
result


rule,reverse LR (NBBO)
TRADE_SIZE_binned,
"(0,1]",59.769889
"(1,3]",62.717707
"(3,5]",63.160616
"(5,11]",64.665735
>11,70.000512


In [23]:
# see p. 30
np.testing.assert_allclose(
    result.iloc[:, 0].tolist(), [59.48, 62.43, 62.83, 64.43, 69.91], atol=0.5
)


In [24]:
combine_results(result_ts, result)


TRADE_SIZE_binned,"(0,1]","(1,3]","(3,5]","(5,11]",>11
rule,,,,,
reverse LR (NBBO),74.59 (14.82),77.93 (15.21),79.66 (16.5),72.35 (7.69),69.33 (-0.67)


### EMO Algorithm

In [25]:
clf = ClassicalClassifier(
    layers=[("trade_size", "ex"), ("emo", "best")],
    random_state=seed,
)
clf.fit(X=X_print.loc[0:1, :], y=X_print["buy_sell"].loc[0:1])
X_print["buy_sell_predicted"] = clf.predict(X_print)
X_print["rule"] = "Tradesize + EMO (NBBO)"

check_robustness("TRADE_SIZE_binned")


rule,Tradesize + EMO (NBBO)
TRADE_SIZE_binned,
"(0,1]",67.820610
"(1,3]",70.778904
"(3,5]",72.683370
"(5,11]",65.987162
>11,63.062609


### Reverse LR Algorithm

In [26]:
clf = ClassicalClassifier(
    layers=[("trade_size", "ex"), ("rev_emo", "best")],
    random_state=seed,
)
clf.fit(X=X_print.loc[0:1, :], y=X_print["buy_sell"].loc[0:1])
X_print["buy_sell_predicted"] = clf.predict(X_print)
X_print["rule"] = "Tradesize + rev. EMO (NBBO)"

check_robustness("TRADE_SIZE_binned")


rule,Tradesize + rev. EMO (NBBO)
TRADE_SIZE_binned,
"(0,1]",68.192083
"(1,3]",71.163909
"(3,5]",73.170740
"(5,11]",66.192528
>11,62.754109


In [27]:
clf = ClassicalClassifier(
    layers=[
        ("trade_size", "ex"),
        ("rev_lr", "best"),
        ("rev_lr", "ex"),
        ("depth", "ex"),
    ],
    random_state=seed,
)
clf.fit(X=X_print.loc[0:1, :], y=X_print["buy_sell"].loc[0:1])
X_print["buy_sell_predicted"] = clf.predict(X_print)
X_print["rule"] = "tradesize + reverse LR (NBBO, ISE) + depth"

result = check_robustness("TRADE_SIZE_binned")
result


rule,"tradesize + reverse LR (NBBO, ISE) + depth"
TRADE_SIZE_binned,
"(0,1]",74.591775
"(1,3]",77.924939
"(3,5]",79.656328
"(5,11]",72.353742
>11,69.329910


In [28]:
# see p. 30
np.testing.assert_allclose(
    result.iloc[:, 0].tolist(), [75.67, 79.04, 80.72, 73.30, 70.24], atol=1.2
)


In [29]:
check_robustness("year_binned")


rule,"tradesize + reverse LR (NBBO, ISE) + depth"
year_binned,
2005-2007,80.902082
2008-2010,79.788060
2011,78.607726
2012,77.368213
2013,75.930256
2014,71.015736
2015,64.010954
2016,66.494553
2017,68.136512
